In [1]:
from typing import BinaryIO
from pulp import *

#list of supplying facilities
facility=['f1','f2','f3']

#list of customers
customers=['c1','c2','c3','c4','c5']

#dmenad requirement at customers
demand={ 'c1':80,
         'c2':270,
         'c3':250,
         'c4':160,
         'c5':180}

#activation cost of facility location
actC={'f1':1000,
      'f2':1000,
      'f3':1000}

#supply limit quantity
supply={'f1':500,
        'f2':500,
        'f3':500}

#cost for transporting from facilty f to customer c
transcost={'f1':{'c1':4, 'c2':5, 'c3':6, 'c4':8, 'c5':10},
           'f2':{'c1':6, 'c2':4, 'c3':3, 'c4':5, 'c5':8},
           'f3':{'c1':9, 'c2':7, 'c3':4, 'c4':3, 'c5':4} }

#creating Lp problem
RA=LpProblem('ResAlloc',LpMinimize)

#creating decision variables, two type of dv are here, one is for quantity and one is for site selection
#continous variable for all possible connection between j and i with service quantity to be transported
routes=[(i,j) for i in facility for j in customers]

ser_var=LpVariable.dicts('service',(facility,customers),0)

#binary variable for site selection at faciltiy 
use_var=LpVariable.dicts('Yi',facility,0,1,LpBinary)

#objective function
RA += lpSum( actC[i]*use_var[i] for i in facility )  + lpSum( ser_var[i][j]*transcost[i][j] for (i,j) in routes)

#constraint1 = getting collected at demand
for j in customers:
        RA += lpSum( ser_var[i][j] for i in facility    ) == demand[j] 

#constraint2 = starting from supplier
for i in facility:
        RA += lpSum( ser_var[i][j] for j in customers  ) <= supply[i]*use_var[i]

#constraint3 = binary limitation
for j in customers:
        for i in facility:
                RA += ser_var[i][j] <= demand[j]*use_var[i] 

#calling solver
RA.solve()
print('status:', LpStatus[RA.status] )

#print dv
#for var in RA.variables():
#        print(var, "=", value(var))

tol=0.001
for i in facility:
        if use_var[i].varValue > tol:
                print('establish facility',i)
      
for v in RA.variables():
        print(v.name, "=", v.varValue)

#print objective 
print("Optimalsol for RA =", value(RA.objective))






status: Optimal
establish facility f2
establish facility f3
Yi_f1 = 0.0
Yi_f2 = 1.0
Yi_f3 = 1.0
service_f1_c1 = 0.0
service_f1_c2 = 0.0
service_f1_c3 = 0.0
service_f1_c4 = 0.0
service_f1_c5 = 0.0
service_f2_c1 = 80.0
service_f2_c2 = 270.0
service_f2_c3 = 150.0
service_f2_c4 = 0.0
service_f2_c5 = 0.0
service_f3_c1 = 0.0
service_f3_c2 = 0.0
service_f3_c3 = 100.0
service_f3_c4 = 160.0
service_f3_c5 = 180.0
Optimalsol for RA = 5610.0
